# install
## you'll need [anaconda](https://www.anaconda.com/download)
## see [ultralytics](https://docs.ultralytics.com/quickstart/)
```bash
conda create -n kibo python=3.11
conda activate kibo
# Install the ultralytics package from PyPI
pip install ultralytics
```
You'll need to install [pytorch](https://pytorch.org/get-started/locally/)
## for windows with nvdia gpu
```bash
conda install cudatooolkit=11.8
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```
## other os or gpu... god bless you

## in google colab

# check installation of pytorch

In [1]:
import torch
import os

print(torch.__version__)
if os.name == 'nt':
    # windows
    print(f'cuda: {torch.cuda.is_available()}')
else:
    # mac
    print(f'mps: {torch.backends.mps.is_available()}')


2.2.2+cu118
cuda: True


# for more detail see [yolo doc](https://docs.ultralytics.com/quickstart/#install-ultralytics)

In [ ]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO("yolov8n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolov8n.pt")

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data="coco128.yaml", epochs=3)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
success = model.export(format="onnx")